#### After mapping each AOI to 2 time windows for each car, we will predict the KPI of the AOI

There are in total 1226 AOIs for HH and 940 for VH.

We have X windows for each AOI based on the filter. 

The weights should be found based on the Two closest points, and the avg. speed that it has at these two points.

The logic is that there are always(*) a second on each side of the AOI. By using the distance and the speed, one can determine 

1 and 1,1

Total distance from 

or 1 and 2

(*) If the speed differs alot between the seconds they could both be on the same side, but that seems to be very unlikely. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
import h5py
from scipy.interpolate import CubicSpline
from copy import deepcopy
from tqdm import tqdm
import re
import csv
from pathlib import Path

In [2]:
import h5py

def load_from_hdf5(filename):

    def unpack_group(group):
        unpacked_data = {}
        for key, item in group.items():
            if isinstance(item, h5py.Group):
                unpacked_data[key] = unpack_group(item)
            else:
                unpacked_data[key] = item[:].tolist()  # Convert dataset to list
        return unpacked_data

    with h5py.File(filename, 'r') as hdf_file:
        loaded_mapping = unpack_group(hdf_file)
    
    return loaded_mapping

# Takes about 40 seconds to load
h5py_mapping_hh = load_from_hdf5("../data/processed/AOI/mapping_hh_time_to_location.hdf5")
h5py_mapping_vh = load_from_hdf5("../data/processed/AOI/mapping_hh_time_to_location.hdf5")


In [68]:
def get_total_distance(data):
    first_values = []

    for trip_name, passes in data.items():
        for pass_name, segments in passes.items():
            for segment_name, values in segments.items():
                if segment_name in ['distance_segment_second_1', 'distance_segment_second_2']:
                    # Extract the first value and add to the list
                    first_values.append(values[0])

    total_distance = np.sum(first_values)
    return total_distance


def time_to_drive_X_meters(speed_kmh, distance_m=10):
    # Convert speed from km/h to m/s
    speed_ms = speed_kmh * (1000 / 3600)

    # Calculate time in seconds
    time_seconds = distance_m / speed_ms
    
    return time_seconds


# Weight functions 
def ln_of_ratio(values, total):
    weight_values = -np.log(values / total)
    norm = np.linalg.norm(weight_values)
    unit_vector = weight_values / norm
    return unit_vector


def ln_of_ratio_sum_to_1(values, total):
    weight_values = -np.log(values / total)
    total = np.sum(weight_values)
    weight_values_1 = weight_values / total
    return weight_values_1


def inverse_of_ratio(values, total):
    weight_values = (values / total)**(-1)
    norm = np.linalg.norm(weight_values)
    unit_vector = weight_values / norm
    return unit_vector


def inverse_of_ratio_sum_to_1(values, total):
    weight_values = (values / total)**(-1)
    total = np.sum(weight_values)
    weight_values_1 = weight_values / total
    return weight_values_1


In [70]:
test = np.array([1, 2, 3])
total = 6.

print(ln_of_ratio(test, total))
print(ln_of_ratio_sum_to_1(test, total))

print(inverse_of_ratio(test, total))
print(inverse_of_ratio_sum_to_1(test, total))



[0.80961586 0.49641369 0.31320217]
[0.5       0.3065736 0.1934264]
[0.85714286 0.42857143 0.28571429]
[0.54545455 0.27272727 0.18181818]


In [18]:
for car in h5py_mapping_hh["0"]:
    print(h5py_mapping_hh["0"][car])
    for trip in h5py_mapping_hh["0"][car]:
        print(h5py_mapping_hh["0"][car][trip]) 
        for distance_segment_second in h5py_mapping_hh["0"][car][trip]:
            print(h5py_mapping_hh["0"][car][trip][distance_segment_second])
            break
        break
    break


{'pass_1': {'distance_segment_second_1': [2.3317746094376914e-05, 0.0, 0.0], 'distance_segment_second_2': [3.658563558098666e-05, 0.0, 1.0]}, 'pass_11': {'distance_segment_second_1': [6.483034333046726e-05, 2.0, 2.0], 'distance_segment_second_2': [9.578165683427985e-05, 2.0, 1.0]}, 'pass_13': {'distance_segment_second_1': [3.2137861824301134e-05, 3.0, 1.0], 'distance_segment_second_2': [4.7965809663709314e-05, 3.0, 0.0]}, 'pass_15': {'distance_segment_second_1': [7.158024588141076e-05, 5.0, 0.0], 'distance_segment_second_2': [8.890653162778374e-05, 5.0, 1.0]}, 'pass_17': {'distance_segment_second_1': [2.3883416921457747e-05, 7.0, 1.0], 'distance_segment_second_2': [6.0817756440021506e-05, 7.0, 0.0]}, 'pass_3': {'distance_segment_second_1': [4.2139430466692684e-05, 10.0, 0.0], 'distance_segment_second_2': [6.319870650188742e-05, 10.0, 1.0]}, 'pass_5': {'distance_segment_second_1': [9.47361156045296e-05, 12.0, 0.0], 'distance_segment_second_2': [9.47361156045296e-05, 12.0, 1.0]}, 'pass_9

In [71]:
mapping_with_weights_hh = h5py_mapping_hh
mapping_with_weights_vh = h5py_mapping_vh
maps = [mapping_with_weights_hh, mapping_with_weights_vh]

new_mapping = {}

for mapping in maps:
    indexes_strings = list(mapping.keys())
    indexes = np.sort([int(x) for x in mapping])
    for index in indexes:
        new_mapping[index] = []
        distances = []
        segments = []
        seconds = []

        total_distance_for_index = get_total_distance(mapping[str(index)])
        # Now go into each individual trip and segment and calculate the weight
        for car in mapping[str(index)]:
            for trip in mapping[str(index)][car]:
                for distance_segment_second in mapping[str(index)][car][trip]:
                    distances.append(mapping[str(index)][car][trip][distance_segment_second][0])
                    segments.append(mapping[str(index)][car][trip][distance_segment_second][1])
                    seconds.append(mapping[str(index)][car][trip][distance_segment_second][2])

        
        weight_ln_ratio = ln_of_ratio_sum_to_1(distances, total_distance_for_index)
        weight_inverse_ratio = inverse_of_ratio_sum_to_1(distances, total_distance_for_index)

        for i in range(len(distances)):
            new_mapping[index].append([distances[i], segments[i], seconds[i], weight_ln_ratio[i], weight_inverse_ratio[i]])


In [75]:
new_mapping[index]

[[8.508740212138433e-05, 4.0, 49.0, 0.06547794769130515, 0.06307755338794452],
 [9.232527915058013e-05, 4.0, 48.0, 0.06347318278045366, 0.05813256346833801],
 [6.600137795377312e-05, 8.0, 88.0, 0.07171537179454744, 0.08131807723336026],
 [6.972502513463056e-05, 8.0, 87.0, 0.07036762292939944, 0.07697530606249184],
 [7.020840886891911e-05, 13.0, 59.0, 0.07019796738324847, 0.07644533235290442],
 [7.021387426553937e-05, 13.0, 60.0, 0.07019605585179835, 0.07643938190414415],
 [9.422030566549493e-05, 20.0, 40.0, 0.06297425113592345, 0.05696335956505656],
 [9.538483107683817e-05,
  22.0,
  43.0,
  0.06267260336149204,
  0.056267910624380094],
 [8.033683081554249e-05, 25.0, 86.0, 0.06688873626929294, 0.06680752894368305],
 [8.780281214380418e-05, 25.0, 85.0, 0.06470651771302419, 0.0611268024213488],
 [8.721455087953374e-05, 27.0, 85.0, 0.06487159470441167, 0.06153910208591807],
 [8.891766387740006e-05, 27.0, 84.0, 0.06439668177105286, 0.06036039315375293],
 [8.506221598201344e-05, 61.0, 45.0,